In [86]:
# импорт зависимостей
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectKBest, f_regression

In [87]:
data = pd.read_excel('data/data_ford_price.xlsx') 
data.head()

,price,year,condition,cylinders,odometer,title_status,transmission,drive,size,lat,long,weather
0,43900,2016,4,6,43500,clean,automatic,4wd,full-size,36.471500,-82.483400,59.0
1,15490,2009,2,8,98131,clean,automatic,4wd,full-size,40.468826,-74.281734,52.0
2,2495,2002,2,8,201803,clean,automatic,4wd,full-size,42.477134,-82.949564,45.0
3,1300,2000,1,8,170305,rebuilt,automatic,4wd,full-size,40.764373,-82.349503,49.0
4,13865,2010,3,8,166062,clean,automatic,4wd,NaN,49.210949,-123.114720,NaN


In [88]:
data = data[['price','year', 'cylinders', 'odometer', 'lat', 'long', 'weather']]
data.dropna(inplace = True)

y = data['price']
x = data.drop(columns='price')

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

In [89]:
# обучение на всех признаках

model = LinearRegression()
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)
 
mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

mape = mean_absolute_percentage_error(y_test, y_predicted)
print('MAPE: %.3f' % mape)

r2_score_metric = r2_score(y_test, y_predicted)
print('r2 score: %.3f' % r2_score_metric)

MAE: 4682.957
MAPE: 166.464
r2 score: 0.608


In [90]:
# метод рекурсивного исключения признаков (RFE)

estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=3, step=1)
selector = selector.fit(X_train, y_train)
 
selector.get_feature_names_out()

array(['year', 'cylinders', 'lat'], dtype=object)

In [91]:
# метод SelectKBest

selector = SelectKBest(f_regression, k=3)
selector.fit(X_train, y_train)
 
selector.get_feature_names_out()

array(['year', 'cylinders', 'odometer'], dtype=object)

In [92]:
# обученение регрессии на первых трёх столбцах, оценка качества модели на тесте

X_train_new_rfe = X_train[['year', 'cylinders', 'lat']]
X_test_new_rfe = X_test[['year', 'cylinders', 'lat']]

model = LinearRegression()
model.fit(X_train_new_rfe, y_train)
y_predicted = model.predict(X_test_new_rfe)
 
mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

mape = mean_absolute_percentage_error(y_test, y_predicted)
print('MAPE: %.3f' % mape)

r2_score_metric = r2_score(y_test, y_predicted)
print('R^2 score: %.3f' % r2_score_metric)

MAE: 5096.570
MAPE: 162.949
R^2 score: 0.576


In [93]:
# обученение регрессии на вторых трёх столбцах, оценка качества модели на тесте

X_train_new_skb = X_train[['year', 'cylinders', 'odometer']]
X_test_new_skb = X_test[['year', 'cylinders', 'odometer']]

model = LinearRegression()
model.fit(X_train_new_skb, y_train)
y_predicted = model.predict(X_test_new_skb)
 
mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

mape = mean_absolute_percentage_error(y_test, y_predicted)
print('MAPE: %.3f' % mape)

r2_score_metric = r2_score(y_test, y_predicted)
print('r2 score: %.3f' % r2_score_metric)

MAE: 4708.946
MAPE: 172.123
r2 score: 0.602


#### Cравнение выбранных метрик. 
Средняя абсолютная ошибка (Mean Absolute Error) интерпретируется следующим образом: это число показывает, насколько в среднем обученная модель ошибается. Чем меньше значение метрики, тем лучше качество модели. Таким образом, у нас есть метрика MAE, рассчитанная для двух моделей линейной регрессии. А именно первый вариант для столбцов-признаков ['year', 'cylinders', 'lat'], полученных методом рекурсивного исключения признаков (RFE), и второй вариант - ['year', 'cylinders', 'odometer'] согласно методу SelectKBest. В первом случае MAE составляет 5096.570, а во втором - 4708.946. Значит, можно говорить о том, что качество второй модели лучше, чем первой, поскольку MAE, полученная на вторых трех признаках, меньше MAE, рассчтанной для модели, обученной на первых трех признаках. Кроме того, метрика R^2, которая показывает, какую долю разнообразия (дисперсии) смогла уловить модель в данных, выше на второй модели (0.602 против 0.576).